# Neural Digit Visualizer

## 0. Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense  # type: ignore

from utils import *

import logging
import warnings

np.set_printoptions(precision=2)
tf.autograph.set_verbosity(0)
tf.random.set_seed(1234)
logging.getLogger("tensorflow").setLevel(logging.ERROR)
warnings.simplefilter(action="ignore", category=FutureWarning)

## 1. Problem Statement

In this project, we will build a neural network to recognize ten handwritten digits, `0-9`. This is a multiclass classification task where one of n choices is selected. Automated handwritten digit recognition is widely used today, from recognizing zip codes (postal codes) on mail envelopes to identifying amounts written on bank checks.

## 2. Dataset

We will start by loading the dataset for this task.

- The first part of the training set is a 400-dimensional vector `X` that contains the input features for the training set:
    - Each training example is a 20x20 pixels grayscale image of the digit.
    - Each pixel is represented by a `1` or a `0` number indicating the intensity at that location.
    - The 20 by 20 grid of pixels is unrolled into a 400-dimensional vector.

- The second part of the training set is a 1-dimensional vector `y` that contains labels for the training set:
    - `y = 0` if the image is of the digit `0`, `y = 4` if the image is of the digit `4` and so on.
    - `y = 10` if the grid is empty.

In [2]:
X, y = load_data()

In [ ]:
print(type(X))
print(type(y))

print(X.shape)
print(y.shape)

## 3. Visualizing the Data

Let's begin by visualizing a subset of the training set.

In the cell below, the code randomly selects 8 rows from `X`, maps each row back to a 20x20 pixels grayscale image and displays the images together.

The label for each image is displayed above the image.

In [ ]:
m, n = X.shape

fig, axes = plt.subplots(2, 4, figsize=(10, 5))
fig.tight_layout(pad=1)

for i, ax in enumerate(axes.flat):
    random_index = np.random.randint(m)

    X_random_reshaped = X[random_index].reshape((20, 20))
    ax.imshow(X_random_reshaped, cmap='gray')

    ax.set_title(y[random_index])
    ax.set_axis_off()

## 4. Model Representation

The neural network we are building is shown in the figure below.

This has the following structure:
- Two dense layers with ReLU activations
- An output layer with a linear activation

<center>
    <img src="./Images/model-representation.png" alt="Model Representation" height="300" />
</center>

## 5. TensorFlow Model Implementation

In [4]:
model = Sequential(
    [
        tf.keras.Input(shape=(400,)),  # type: ignore
        Dense(units=20, activation="relu", name="L1"),
        Dense(units=16, activation="relu", name="L2"),
        Dense(units=11, activation="linear", name="L3"),
    ],
    name="ndv_model",
)

In [ ]:
model.summary()

This is where the softmax activation function is applied:

In [ ]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # type: ignore
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # type: ignore
)

history = model.fit(X, y, epochs=45)

In [6]:
model_name = "ndv_model"
model.save(f"{model_name}.keras")

In [13]:
model = tf.keras.models.load_model(f"{model_name}.keras")  # type: ignore

In [14]:
[L1, L2, L3] = model.layers

W1, b1 = L1.get_weights()
W2, b2 = L2.get_weights()
W3, b3 = L3.get_weights()

In [ ]:
np.set_printoptions(threshold=np.inf) # type: ignore

print(W1)

In [ ]:
plot_loss_tf(history)

## 6. Making Predictions

To make a prediction, we will use the model `predict` method. Below, `X[0]` contains an image of a seven.

In [ ]:
image_of_seven = X[0]

display_digit(image_of_seven)

prediction = model.predict(image_of_seven.reshape(1, 400))

print(f"Predicting a 'Seven': {prediction}")
print(f"Largest prediction index: {np.argmax(prediction)}")

The largest output is prediction[7], indicating the predicted digit is a `7`. If the problem only requires a selection, that is sufficient. Use NumPy `argmax` to select it. If the problem requires a probability, a softmax is required:

In [ ]:
prediction_prb = tf.nn.softmax(prediction)
probabilities = " ".join([f"{prob:2.3f}" for prob in prediction_prb[0]])

print(f"Predicting a 'Seven' probability vector: {probabilities}")
print(f"Total of predictions: {np.sum(prediction_prb):0.3f}")

In [ ]:
yhat = np.argmax(prediction_prb)
print(f"np.argmax(prediction_prb): {yhat}")

Let's compare the predictions `vs` the labels for a random sample of 8 digits.

In [ ]:
m, n = X.shape

fig, axes = plt.subplots(2, 4, figsize=(10, 5))
fig.tight_layout(pad=1)

for i, ax in enumerate(axes.flat):
    random_index = np.random.randint(m)

    X_random_reshaped = X[random_index].reshape((20, 20))

    ax.imshow(X_random_reshaped, cmap='gray')

    prediction = model.predict(X[random_index].reshape(1, 400))
    prediction_prb = tf.nn.softmax(prediction)
    yhat = np.argmax(prediction_prb)

    ax.set_title(f"{y[random_index]}, {yhat} -> {np.max(prediction_prb)*100:2.3f}%", fontsize=10)
    ax.set_axis_off()

plt.show()

In [ ]:
print(f"{display_errors(model, X, y, True)} errors out of {len(X)} images")

<br />

## Thanks for making it this far! Not everyone does!

Now that you understand how we trained this model, feel free to check the other sections to see how we built and simulated this neural digit visualizer!